## Data cleaning and preparation

In [1]:
import sys
import pandas as pd
import numpy as np

Durante el curso de análisis y modelado de datos, se dedica una cantidad significativa
de tiempo a la preparación de datos: carga, limpieza, transformación y reorganización.
A menudo se informa que tales tareas ocupan el 80% o más del tiempo de un analista.
A veces, la forma en que los datos se almacenan en archivos o bases de datos no están
en el formato correcto para una tarea en particular. Muchos invetigadores optan por
realizar un procesamiento _ad hoc_ de una forma a otra utilizando un lenguage de
programación de propósito general, como Python, Perl, R o Java, o herramientas de
procesamiento de texto Unix como `sed` o `awk`. Afortunadamente, Pandas, junto con las
características integradas de Python, le proporcionan un conjunto de herramientas de
alto nivel, flexibles y rápidas para permitirle manipular los datos en la forma correcta.

### Manejo de datos faltantes
Los datos faltantes ocurren comúnmente en muchas aplicaciones de análisis de datos.
Uno de los objetivos de Pandas es hacer que trabajar con datos faltantes sea lo más
indoloro posible. Por ejemplo, todas las estadísticas descriptivas sobre objetos
Pandas excluye los datos faltantes de forma predeterminada.

La forma en que los datos faltantes se representan en los objetos Pandas es algo
imperfecta, pero es suficiente para la mayoría del uso del mundo real. Para datos
con dtype `float64`, pandas utiliza el valor de punto flotante `NaN` (no es un número)
para representar los datos faltantes.

Llamamos a esto un valor centinela: cuando está presente, indica un valor faltante (o nulo):

In [5]:
float_data = pd.Series([1.2, -3.5, np.nan, 0])
float_data

0    1.2
1   -3.5
2    NaN
3    0.0
dtype: float64

In [6]:
# isna devuelve una serie booleana con True en valores nulos
float_data.isna()

0    False
1    False
2     True
3    False
dtype: bool

En Pandas, hemos adaptado una convención utilizada en el lenguaje de programación R
refiréndonos a los datos faltantes como NA, que significa _no disponible_. En las
aplicaciones estadísticas, los datos de NA pueden ser datos que no existen o que existen,
pero no se observaron. Al limpiar los datos para el análisis, a menudos es importante
hacer análisis sobre los datos faltantes para identificar problemas de recopilación de
datos o posibles sesgos en los datos causados por los datos faltantes.

In [7]:
# el valor None también se trata como NA
string_data = pd.Series(["aardvark", np.nan, None, "aguacate"])
string_data

0    aardvark
1         NaN
2        None
3    aguacate
dtype: object

In [8]:
string_data.isna()

0    False
1     True
2     True
3    False
dtype: bool

In [9]:
float_data = pd.Series([1, 2, None], dtype='float64')
float_data

0    1.0
1    2.0
2    NaN
dtype: float64

In [10]:
float_data.isna()

0    False
1    False
2     True
dtype: bool

In [13]:
float_data.dropna() # evita los valores NA

0    1.0
1    2.0
dtype: float64

In [14]:
float_data.fillna("NULL") # sustituye valores faltantes por otro valor

0     1.0
1     2.0
2    NULL
dtype: object

In [15]:
float_data.notna() # True en los no faltantes

0     True
1     True
2    False
dtype: bool

### Filtrar datos faltantes
Hay algunas formas de filtrar los datos faltantes. Si bien siempre tiene la opción de
hacerlo a mano usando `pd.isna` e indexación booleana, `dropna` puede ser útil.
En una serie, devuelve la serie con solo los datos no nulos y los valores de índice:

In [16]:
data = pd.Series([1, np.nan, 3.5, np.nan, 7])
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

In [17]:
# es lo mismo que hacer
data[data.notna()]

0    1.0
2    3.5
4    7.0
dtype: float64

Con los objetos DataFrame, hay diferentes formas de eliminar los datos faltantes. Es
posible que desee soltar filas o columnas que son todas NA, o solo aquellas filas o
columnas que contienen NA, `dropna` por defecto, suelta cualquier fila que contenga
un valor faltante:

In [18]:
data = pd.DataFrame([[1., 6.5, 3.],
                    [1., np.nan, np.nan],
                    [np.nan, np.nan, np.nan],
                    [np.nan, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [19]:
data.dropna()

,0,1,2
0,1.0,6.5,3.0


Pasando `how="all"` solo soltará filas que sean todas NA:

In [20]:
data.dropna(how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


Para soltar columnas `axis="columns"`:

In [21]:
data[4] = np.nan
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [22]:
data.dropna(axis="columns", how="all")

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


Supongamos que desea mantener solo filas que contengan como máximo un cierto número de
observaciones faltantes. Puede indicar esto con el argumento `thresh`:

In [26]:
df = pd.DataFrame(np.random.standard_normal((7, 3)))
df

,0,1,2
0,-1.638638,1.105508,-1.027507
1,-0.983494,-0.965537,-0.358571
2,-1.256609,-0.141638,-1.055511
3,0.984909,-2.335164,0.044647
4,-1.963444,-0.402822,0.228176
5,-1.205198,0.772947,0.678766
6,-0.800833,-0.330168,-0.965351


In [27]:
df.iloc[:4, 1] = np.nan

In [28]:
df

,0,1,2
0,-1.638638,NaN,-1.027507
1,-0.983494,NaN,-0.358571
2,-1.256609,NaN,-1.055511
3,0.984909,NaN,0.044647
4,-1.963444,-0.402822,0.228176
5,-1.205198,0.772947,0.678766
6,-0.800833,-0.330168,-0.965351


In [29]:
df.iloc[:2, 2] = np.nan
df

,0,1,2
0,-1.638638,NaN,NaN
1,-0.983494,NaN,NaN
2,-1.256609,NaN,-1.055511
3,0.984909,NaN,0.044647
4,-1.963444,-0.402822,0.228176
5,-1.205198,0.772947,0.678766
6,-0.800833,-0.330168,-0.965351


In [32]:
df.dropna()

,0,1,2
4,-1.963444,-0.402822,0.228176
5,-1.205198,0.772947,0.678766
6,-0.800833,-0.330168,-0.965351


In [33]:
df.dropna(thresh=2)

,0,1,2
2,-1.256609,NaN,-1.055511
3,0.984909,NaN,0.044647
4,-1.963444,-0.402822,0.228176
5,-1.205198,0.772947,0.678766
6,-0.800833,-0.330168,-0.965351


### Rellenando datos faltentes